In [1]:
# Per patient - option to select by drug or visit

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import datetime as dt
import typing as T

pd.set_option("display.max_columns", None)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
def load_csv(filename: str, sep: str, parse_dates: list[str]) -> pd.DataFrame:
    df = pd.read_csv(
        filepath_or_buffer=filename,
        sep=sep,
        parse_dates=parse_dates,
        date_format={col: "%Y%m%d" for col in parse_dates},
    )
    return df

In [3]:
def merge_dataset(left_df: pd.DataFrame, left_on: str, right_df: pd.DataFrame, right_on: str, prefixes: tuple[str, str], how: str = "left") -> pd.DataFrame:
    left_df = left_df.copy()
    if prefixes[0] is not None:
        left_df.columns = [f"{prefixes[0]}_{col}" for col in left_df.columns]
        left_on = f"{prefixes[0]}_{left_on}"

    right_df = right_df.copy()
    if prefixes[1] is not None:
        right_df.columns = [f"{prefixes[1]}_{col}" for col in right_df.columns]
        right_on = f"{prefixes[1]}_{right_on}"

    df = pd.merge(left=left_df, left_on=left_on, right=right_df, right_on=right_on, how=how)
    return df


def preprocess_drug_strength(drug_df: pd.DataFrame, concept_df: pd.DataFrame, keep_only_valid_unit: bool = False) -> pd.DataFrame:

    strength_df = drug_df.copy()
    # CHECK VALIDITY DATES
    # Invalid reason: D (deleted), U (replaced with an update), NULL (valid_end_date has the default value)
    strength_df["invalid_drug"] = strength_df["valid_end_date"].apply(lambda x: x < dt.datetime.now())

    assert len(strength_df["invalid_reason"].dropna()) == strength_df["invalid_drug"].sum()

    # ADD CONCEPT DEFINITION
    unit_concept_df = concept_df[concept_df["domain_id"] == "Unit"].reset_index(drop=True)
    # Keep only concepts that are valid
    if keep_only_valid_unit:
        unit_concept_df = unit_concept_df[unit_concept_df["valid_end_date"] >= dt.datetime.now()].reset_index(drop=True)

    # drug_concept_df = concept_df[concept_df["domain_id"] == "Drug"].reset_index(drop=True)
    drug_concept_df = concept_df.copy()

    # MERGE CONCEPTS
    # 1. Amount concept
    df = merge_dataset(
        left_df=strength_df,
        right_df=unit_concept_df,
        left_on="amount_unit_concept_id",
        right_on="concept_id",
        prefixes=(None, "amount"),
        how="left",
    )
    # 2. Numerator concept
    df = merge_dataset(
        left_df=df,
        right_df=unit_concept_df,
        left_on="numerator_unit_concept_id",
        right_on="concept_id",
        prefixes=(None, "numerator"),
        how="left",
    )
    # 3. Denominator concept
    df = merge_dataset(
        left_df=df,
        right_df=unit_concept_df,
        left_on="denominator_unit_concept_id",
        right_on="concept_id",
        prefixes=(None, "denominator"),
        how="left",
    )
    # 4. Ingredient concept
    df = merge_dataset(
        left_df=df,
        right_df=drug_concept_df,
        left_on="ingredient_concept_id",
        right_on="concept_id",
        prefixes=(None, "ingredient")
    )
    return df

In [4]:
def refining_drug_exposure(drug_df: pd.DataFrame, concept_df: pd.DataFrame) -> pd.DataFrame:
    exposure_df = drug_df.copy()

    # drug_concept_df = concept_df[concept_df["domain_id"] == "Drug"].reset_index(drop=True)
    drug_concept_df = concept_df.copy()

    # MERGE CONCEPTS
    # 1. Drug
    df = merge_dataset(
        left_df=exposure_df,
        left_on="drug_concept_id",
        right_df=drug_concept_df,
        right_on="concept_id",
        prefixes=(None, "drug"),
        how="left",
    )
    # 2. Drug type
    df = merge_dataset(
        left_df=df,
        left_on="drug_type_concept_id",
        right_df=drug_concept_df,
        right_on="concept_id",
        prefixes=(None, "drug_type"),
        how="left",
    )
    # 3. Route
    df = merge_dataset(
        left_df=df,
        left_on="route_concept_id",
        right_df=drug_concept_df,
        right_on="concept_id",
        prefixes=(None, "route"),
        how="left",
    )

    return df

In [5]:
def compute_drug_dose(row: pd.Series) -> float:
    # 1. Tablets and other fixed amount formulations
    # DRUG_STRENGTH.denominator_unit_concept_id = empty
    if row["denominator_unit_concept_id"] is None:
        dose = row["quantity"] * row["amount_value"]
        dose_lit = f"{dose} {row['denominator_concept_code']}"
    # 2. Puffs of an inhaler
    # Denominator unit is {actuat}
    elif row["denominator_unit_concept_id"] == 45744809:
        dose = row["quantity"] * row["numerator_value"]
        dose_lit = f"{dose} {row['numerator_concept_code']}"
    # Denominator is either mL or mg
    elif row["denominator_unit_concept_id"] in [8587, 8576]:  # [mL, mg]
        # 3. Quantified Drugs which are formulated as a concentration mL or mg, but
        # denominator_value might be different from 1
        if row["denominator_value"] != 1:
            dose = row["quantity"] * row["numerator_value"]
            dose_lit = f"{dose} {row['numerator_concept_code']}"
        # 4. Drugs with the total amount provided in quantity
        # NLP analysis is needed to analyze concept name (get concentration, content, units, etc.)
        # TO CHECK 1: check that quantity is expressed in mL or mL and not in oz
        # TO CHECK 2: check denominator unit e.g. g vs mg (factor 1000)
        else:
            dose = row["quantity"] * row["numerator_value"]
            dose_lit = f"{dose} {row['numerator_concept_code']}"
    # 6. Drugs with the active ingredient released over time
    elif row["denominator_unit_concept_id"] == 8505:
        dose = row["numerator_value"]
        dose_lit = f"{dose} {row['numerator_concept_code']} / {row['denominator_concept_code']}"
    # 5. Compounded drugs
    # TODO: NLP analysis needed to get different ingredients
    else:
        dose = None
        dose_lit = None

    return dose, dose_lit

In [6]:
def get_dataframe_per_patient(df: pd.DataFrame, patient_id: int) -> pd.DataFrame:
    patient_df = df[df["person_id"] == patient_id].reset_index(drop=True)
    return patient_df

def get_dataframe_per_visit(df: pd.DataFrame, patient_id: int, visit_id: int) -> pd.DataFrame:
    visit_df = df[(df["person_id"] == patient_id) & (df["visit_occurrence_id"] == visit_id)].reset_index(drop=True)
    return visit_df

def get_unique_patient_id(df: pd.DataFrame) -> list:
    return df["person_id"].unique().tolist()

def get_unique_visit_id(df: pd.DataFrame) -> list:
    return df["visit_occurrence_id"].unique().tolist()

def get_trend(value: float) -> int:
    if value > 0:
        return 1
    elif value < 0:
        return -1
    else:
        return 0

In [7]:
concept_file = "../data/vocabulary_download_v5_all/CONCEPT.csv"
drug_strength_file = "../data/vocabulary_download_v5_all/DRUG_STRENGTH.csv"
drug_exposure_file = "../data/drug_exposure.csv"

concept_df = load_csv(filename=concept_file, sep="\t", parse_dates=["valid_start_date", "valid_end_date"])
drug_strength_df = load_csv(filename=drug_strength_file, sep="\t", parse_dates=["valid_start_date", "valid_end_date"])
drug_exposure_df = pd.read_csv(drug_exposure_file)

/var/folders/5t/vb6260nn0dd8rm99r3ls2p8h0000gp/T/ipykernel_26007/1270663440.py:2: DtypeWarning: Columns (6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(
/var/folders/5t/vb6260nn0dd8rm99r3ls2p8h0000gp/T/ipykernel_26007/1270663440.py:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


In [8]:
drug_exposure_df["drug_exposure_id"].nunique()
drug_exposure_df["person_id"].nunique()
drug_exposure_df["visit_occurrence_id"].nunique()

18229

100

251

In [9]:
drug_exposure_df.head(1)

,drug_exposure_id,person_id,drug_concept_id,drug_exposure_start_date,drug_exposure_start_datetime,drug_exposure_end_date,drug_exposure_end_datetime,verbatim_end_date,drug_type_concept_id,stop_reason,refills,quantity,days_supply,sig,route_concept_id,lot_number,provider_id,visit_occurrence_id,visit_detail_id,drug_source_value,drug_source_concept_id,route_source_value,dose_unit_source_value
0,294884377115777655,1741351032930224901,40166274,2177-07-16,2177-07-16 22:00:00,2177-07-17,2177-07-17 21:00:00,NaN,32838,NaN,NaN,0.005,NaN,NaN,4142048,NaN,NaN,3736965967695233281,NaN,00002751001,45144375,SC,VIAL


In [27]:
def preprocess_drug_exposure(df: pd.DataFrame) -> pd.DataFrame:
    new_df = df.copy()
    columns = ["drug_exposure_id", "person_id", "visit_occurrence_id"]
    columns = [col for col in columns if col in new_df.columns]
    for col in columns:
        unique_values = new_df[col].unique()
        map_values = {val: f"P{i+1:06d}" for i, val in enumerate(unique_values)}

        new_df[col] = new_df[col].map(map_values)
    return new_df

In [10]:
strength_df = preprocess_drug_strength(drug_df=drug_strength_df, concept_df=concept_df)
exposure_df = refining_drug_exposure(drug_df=drug_exposure_df, concept_df=concept_df)

final_df = pd.merge(left=exposure_df, left_on="drug_concept_id", right=strength_df, right_on="drug_concept_id", how="left")
final_df["dose"] = final_df.apply(lambda x: compute_drug_dose(x), axis=1)

In [11]:
persons = get_unique_patient_id(df=final_df)

patient_df = get_dataframe_per_patient(df=final_df, patient_id=persons[0])

visits = get_unique_visit_id(df=patient_df)

visit_df = get_dataframe_per_visit(df=patient_df, patient_id=persons[0], visit_id=visits[0])

In [12]:
patient_df.head()

,drug_exposure_id,person_id,drug_concept_id,drug_exposure_start_date,drug_exposure_start_datetime,drug_exposure_end_date,drug_exposure_end_datetime,verbatim_end_date,drug_type_concept_id,stop_reason,refills,quantity,days_supply,sig,route_concept_id,lot_number,provider_id,visit_occurrence_id,visit_detail_id,drug_source_value,drug_source_concept_id,route_source_value,dose_unit_source_value,drug_concept_name,drug_domain_id,drug_vocabulary_id,drug_concept_class_id,drug_standard_concept,drug_concept_code,drug_valid_start_date,drug_valid_end_date,drug_invalid_reason,drug_type_concept_name,drug_type_domain_id,drug_type_vocabulary_id,drug_type_concept_class_id,drug_type_standard_concept,drug_type_concept_code,drug_type_valid_start_date,drug_type_valid_end_date,drug_type_invalid_reason,route_concept_name,route_domain_id,route_vocabulary_id,route_concept_class_id,route_standard_concept,route_concept_code,route_valid_start_date,route_valid_end_date,route_invalid_reason,ingredient_concept_id,amount_value,amount_unit_concept_id,numerator_value,numerator_unit_concept_id,denominator_value,denominator_unit_concept_id,box_size,valid_start_date,valid_end_date,invalid_reason,invalid_drug,amount_concept_id,amount_concept_name,amount_domain_id,amount_vocabulary_id,amount_concept_class_id,amount_standard_concept,amount_concept_code,amount_valid_start_date,amount_valid_end_date,amount_invalid_reason,numerator_concept_id,numerator_concept_name,numerator_domain_id,numerator_vocabulary_id,numerator_concept_class_id,numerator_standard_concept,numerator_concept_code,numerator_valid_start_date,numerator_valid_end_date,numerator_invalid_reason,denominator_concept_id,denominator_concept_name,denominator_domain_id,denominator_vocabulary_id,denominator_concept_class_id,denominator_standard_concept,denominator_concept_code,denominator_valid_start_date,denominator_valid_end_date,denominator_invalid_reason,ingredient_concept_name,ingredient_domain_id,ingredient_vocabulary_id,ingredient_concept_class_id,ingredient_standard_concept,ingredient_concept_code,ingredient_valid_start_date,ingredient_valid_end_date,ingredient_invalid_reason,dose
0,294884377115777655,1741351032930224901,40166274,2177-07-16,2177-07-16 22:00:00,2177-07-17,2177-07-17 21:00:00,NaN,32838,NaN,NaN,0.005,NaN,NaN,4142048,NaN,NaN,3736965967695233281,NaN,00002751001,45144375,SC,VIAL,insulin lispro 100 UNT/ML Injectable Solution ...,Drug,RxNorm,Branded Drug,S,865098,2009-10-04,2099-12-31,NaN,EHR prescription,Type Concept,Type Concept,Type Concept,S,OMOP4976911,2020-08-20,2099-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,1550023.0,NaN,NaN,100.0,8510.0,NaN,8587.0,NaN,1970-01-01,2099-12-31,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,8510.0,unit,Unit,UCUM,Unit,S,[U],1970-01-01,2099-12-31,NaN,8587.0,milliliter,Unit,UCUM,Unit,S,mL,1970-01-01,2099-12-31,NaN,insulin lispro,Drug,RxNorm,Ingredient,S,86009,1970-01-01,2099-12-31,NaN,"(0.5, 0.5 [U])"
1,-3609243742606366340,1741351032930224901,40166274,2177-07-17,2177-07-17 19:00:00,2177-07-18,2177-07-18 18:00:00,NaN,32838,NaN,NaN,0.014,NaN,NaN,4142048,NaN,NaN,3736965967695233281,NaN,00002751001,45144375,SC,VIAL,insulin lispro 100 UNT/ML Injectable Solution ...,Drug,RxNorm,Branded Drug,S,865098,2009-10-04,2099-12-31,NaN,EHR prescription,Type Concept,Type Concept,Type Concept,S,OMOP4976911,2020-08-20,2099-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,1550023.0,NaN,NaN,100.0,8510.0,NaN,8587.0,NaN,1970-01-01,2099-12-31,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,8510.0,unit,Unit,UCUM,Unit,S,[U],1970-01-01,2099-12-31,NaN,8587.0,milliliter,Unit,UCUM,Unit,S,mL,1970-01-01,2099-12-31,NaN,insulin lispro,Drug,RxNorm,Ingredient,S,86009,1970-01-01,2099-12-31,NaN,"(1.4000000000000001, 1.4000000000000001 [U])"
2,-6865345241721388581,1741351032930224901,40166274,2177-07-15,2177-07-15 19:00:00,2177-07-16,2177-07-16 18:00:00,NaN,32838,NaN,NaN,0.012,NaN,NaN,4142048,NaN,NaN,3736965967695233281,NaN,00002751001,45144375,SC,VIAL,insulin lispro 100 UNT/ML Injectable Solutio

In [13]:
import datetime as dt

In [14]:
patient_df["drug_exposure_start_date"] = pd.to_datetime(patient_df["drug_exposure_start_date"])
patient_df["drug_exposure_end_date"] = pd.to_datetime(patient_df["drug_exposure_end_date"])

In [15]:
patient_df[["drug_exposure_start_date", "drug_exposure_end_date"]].describe()

,drug_exposure_start_date,drug_exposure_end_date
count,493,493
mean,2178-10-20 20:12:10.223124480,2178-10-22 21:39:47.829614592
min,2177-07-14 00:00:00,2177-07-15 00:00:00
25%,2177-09-04 00:00:00,2177-09-05 00:00:00
50%,2178-09-29 00:00:00,2178-10-02 00:00:00
75%,2179-03-30 00:00:00,2179-04-01 00:00:00
max,2182-05-03 00:00:00,2182-05-03 00:00:00


In [20]:
patient_df.shape
visit_df.shape

(493, 102)

(493, 105)

In [24]:
visit_df.tail()

,drug_exposure_id,person_id,drug_concept_id,drug_exposure_start_date,drug_exposure_start_datetime,drug_exposure_end_date,drug_exposure_end_datetime,verbatim_end_date,drug_type_concept_id,stop_reason,refills,quantity,days_supply,sig,route_concept_id,lot_number,provider_id,visit_occurrence_id,visit_detail_id,drug_source_value,drug_source_concept_id,route_source_value,dose_unit_source_value,drug_concept_name,drug_domain_id,drug_vocabulary_id,drug_concept_class_id,drug_standard_concept,drug_concept_code,drug_valid_start_date,drug_valid_end_date,drug_invalid_reason,drug_type_concept_name,drug_type_domain_id,drug_type_vocabulary_id,drug_type_concept_class_id,drug_type_standard_concept,drug_type_concept_code,drug_type_valid_start_date,drug_type_valid_end_date,drug_type_invalid_reason,route_concept_name,route_domain_id,route_vocabulary_id,route_concept_class_id,route_standard_concept,route_concept_code,route_valid_start_date,route_valid_end_date,route_invalid_reason,ingredient_concept_id,amount_value,amount_unit_concept_id,numerator_value,numerator_unit_concept_id,denominator_value,denominator_unit_concept_id,box_size,valid_start_date,valid_end_date,invalid_reason,invalid_drug,amount_concept_id,amount_concept_name,amount_domain_id,amount_vocabulary_id,amount_concept_class_id,amount_standard_concept,amount_concept_code,amount_valid_start_date,amount_valid_end_date,amount_invalid_reason,numerator_concept_id,numerator_concept_name,numerator_domain_id,numerator_vocabulary_id,numerator_concept_class_id,numerator_standard_concept,numerator_concept_code,numerator_valid_start_date,numerator_valid_end_date,numerator_invalid_reason,denominator_concept_id,denominator_concept_name,denominator_domain_id,denominator_vocabulary_id,denominator_concept_class_id,denominator_standard_concept,denominator_concept_code,denominator_valid_start_date,denominator_valid_end_date,denominator_invalid_reason,ingredient_concept_name,ingredient_domain_id,ingredient_vocabulary_id,ingredient_concept_class_id,ingredient_standard_concept,ingredient_concept_code,ingredient_valid_start_date,ingredient_valid_end_date,ingredient_invalid_reason,dose,quantity_diff,quantity_diff_percentage,quantity_diff_trend
488,1655312397065359440,1741351032930224901,991764,2179-03-31,2179-03-31 11:00:00,2179-04-01,2179-04-01 10:00:00,NaN,32838,NaN,NaN,2.0,NaN,NaN,4132161,NaN,NaN,1448527175232053560,NaN,64980010401,44887654,PO,TAB,"potassium phosphate 155 MG / sodium phosphate,...",Drug,RxNorm,Branded Drug,S,656742,2006-11-19,2099-12-31,NaN,EHR prescription,Type Concept,Type Concept,Type Concept,S,OMOP4976911,2020-08-20,2099-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,991710.0,852.0,8576.0,NaN,NaN,NaN,NaN,NaN,1970-01-01,2099-12-31,NaN,False,8576.0,milligram,Unit,UCUM,Unit,S,mg,1970-01-01,2099-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,"sodium phosphate, dibasic",Drug,RxNorm,Ingredient,S,236719,1970-01-01,2099-12-31,NaN,"(None, None)",0.0,0.0,0
489,1655312397065359440,1741351032930224901,991764,2179-03-31,2179-03-31 11:00:00,2179-04-01,2179-04-01 10:00:00,NaN,32838,NaN,NaN,2.0,NaN,NaN,4132161,NaN,NaN,1448527175232053560,NaN,64980010401,44887654,PO,TAB,"potassium phosphate 155 MG / sodium phosphate,...",Drug,RxNorm,Branded Drug,S,656742,2006-11-19,2099-12-31,NaN,EHR prescription,Type Concept,Type Concept,Type Concept,S,OMOP4976911,2020-08-20,2099-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,19027362.0,155.0,8576.0,NaN,NaN,NaN,NaN,NaN,1970-01-01,2099-12-31,NaN,False,8576.0,milligram,Unit,UCUM,Unit,S,mg,1970-01-01,2099-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,potassium phosphate,Drug,RxNorm,Ingredient,S,34322,1970-01-01,2099-12-31,NaN,"(None, None)",0.0,0.0,0
490,7964888222981337738,1741351032930224901,991764,2179-04-01,2179-04-01 13:00:00,2179-04-02,2179-04-02 12:00:00,NaN,32838,NaN,NaN,2.0,NaN,NaN,4132161,NaN,NaN,1448527175232053560,NaN,64980010401,44887654,PO,TAB,"potassium phosphate 155 MG / so

In [23]:
visit_df["drug_concept_name"].nunique()

99

In [25]:
visit_df = patient_df.copy()
visit_df["drug_concept_id"] = visit_df["drug_concept_id"].astype(str)
visit_df = visit_df.sort_values(by=["drug_concept_id", "drug_exposure_start_datetime"]).reset_index(drop=True)
# TODO: Diff by drug_concept_id
visit_df["quantity_diff"] = visit_df.groupby("drug_concept_id")["quantity"].diff()
visit_df["quantity_diff_percentage"] = visit_df.apply(lambda x: round(x["quantity_diff"]/x["quantity"]*100, 2) if x["quantity"] > 0 else 100, axis=1)
visit_df["quantity_diff_trend"] = visit_df["quantity_diff"].apply(get_trend)

fig = px.timeline(
    visit_df,
    x_start="drug_exposure_start_datetime",
    x_end="drug_exposure_end_datetime",
    y="drug_concept_name",
    # color="quantity_diff_trend",
    # color_continuous_scale="bluered",
    # opacity=0.5,
)
fig.show()

In [58]:
visit_df["drug_concept_id"] = visit_df["drug_concept_id"].astype(str)
visit_df = visit_df.sort_values(by=["drug_concept_id", "drug_exposure_start_datetime"]).reset_index(drop=True)
# TODO: Diff by drug_concept_id
visit_df["quantity_diff"] = visit_df.groupby("drug_concept_id")["quantity"].diff()
visit_df["quantity_diff_percentage"] = visit_df.apply(lambda x: round(x["quantity_diff"]/x["quantity"]*100, 2) if x["quantity"] > 0 else 100, axis=1)
visit_df["quantity_diff_trend"] = visit_df["quantity_diff"].apply(get_trend)

In [44]:
drug_exposure_df[drug_exposure_df["drug_concept_id"] == 1596977]["quantity"].describe()

count    99.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
Name: quantity, dtype: float64

In [39]:
visit_df[visit_df["drug_concept_id"] == "19040187"]

,level_0,index,drug_exposure_id,person_id,drug_concept_id,drug_exposure_start_date,drug_exposure_start_datetime,drug_exposure_end_date,drug_exposure_end_datetime,verbatim_end_date,drug_type_concept_id,stop_reason,refills,quantity,days_supply,sig,route_concept_id,lot_number,provider_id,visit_occurrence_id,visit_detail_id,drug_source_value,drug_source_concept_id,route_source_value,dose_unit_source_value,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason,quantity_diff,quantity_diff_percentage,quantity_diff_trend
36,61,61,2660403691083715582,1741351032930224901,19040187,2177-07-15,2177-07-15 04:00:00,2177-07-15,2177-07-15 20:00:00,NaN,32838,NaN,NaN,3.0,NaN,NaN,4132161,NaN,NaN,3736965967695233281,NaN,00456066270,45215516,PO,PKT,19040187.0,potassium chloride 1.33 MEQ/ML Oral Solution [...,Drug,RxNorm,Branded Drug,S,208958,1970-01-01,2099-12-31,NaN,NaN,NaN,0
37,62,62,2489702615899122656,1741351032930224901,19040187,2177-07-15,2177-07-15 04:00:00,2177-07-15,2177-07-15 20:00:00,NaN,32838,NaN,NaN,2.0,NaN,NaN,4132161,NaN,NaN,3736965967695233281,NaN,00456066270,45215516,PO,PKT,19040187.0,potassium chloride 1.33 MEQ/ML Oral Solution [...,Drug,RxNorm,Branded Drug,S,208958,1970-01-01,2099-12-31,NaN,-1.0,-50.0,-1
38,63,63,6451066140159016571,1741351032930224901,19040187,2177-07-15,2177-07-15 04:00:00,2177-07-15,2177-07-15 20:00:00,NaN,32838,NaN,NaN,4.0,NaN,NaN,4132161,NaN,NaN,3736965967695233281,NaN,00456066270,45215516,PO,PKT,19040187.0,potassium chloride 1.33 MEQ/ML Oral Solution [...,Drug,RxNorm,Branded Drug,S,208958,1970-01-01,2099-12-31,NaN,2.0,50.0,1


In [60]:
visit_df[visit_df["drug_concept_id"] == "19049684"]

,drug_exposure_id,person_id,drug_concept_id,drug_exposure_start_date,drug_exposure_start_datetime,drug_exposure_end_date,drug_exposure_end_datetime,verbatim_end_date,drug_type_concept_id,stop_reason,refills,quantity,days_supply,sig,route_concept_id,lot_number,provider_id,visit_occurrence_id,visit_detail_id,drug_source_value,drug_source_concept_id,route_source_value,dose_unit_source_value,drug_concept_name,drug_domain_id,drug_vocabulary_id,drug_concept_class_id,drug_standard_concept,drug_concept_code,drug_valid_start_date,drug_valid_end_date,drug_invalid_reason,drug_type_concept_name,drug_type_domain_id,drug_type_vocabulary_id,drug_type_concept_class_id,drug_type_standard_concept,drug_type_concept_code,drug_type_valid_start_date,drug_type_valid_end_date,drug_type_invalid_reason,route_concept_name,route_domain_id,route_vocabulary_id,route_concept_class_id,route_standard_concept,route_concept_code,route_valid_start_date,route_valid_end_date,route_invalid_reason,ingredient_concept_id,amount_value,amount_unit_concept_id,numerator_value,numerator_unit_concept_id,denominator_value,denominator_unit_concept_id,box_size,valid_start_date,valid_end_date,invalid_reason,invalid_drug,amount_concept_id,amount_concept_name,amount_domain_id,amount_vocabulary_id,amount_concept_class_id,amount_standard_concept,amount_concept_code,amount_valid_start_date,amount_valid_end_date,amount_invalid_reason,numerator_concept_id,numerator_concept_name,numerator_domain_id,numerator_vocabulary_id,numerator_concept_class_id,numerator_standard_concept,numerator_concept_code,numerator_valid_start_date,numerator_valid_end_date,numerator_invalid_reason,denominator_concept_id,denominator_concept_name,denominator_domain_id,denominator_vocabulary_id,denominator_concept_class_id,denominator_standard_concept,denominator_concept_code,denominator_valid_start_date,denominator_valid_end_date,denominator_invalid_reason,ingredient_concept_name,ingredient_domain_id,ingredient_vocabulary_id,ingredient_concept_class_id,ingredient_standard_concept,ingredient_concept_code,ingredient_valid_start_date,ingredient_valid_end_date,ingredient_invalid_reason,dose,quantity_diff,quantity_diff_percentage,quantity_diff_trend
46,P000037,P000000,19049684,2177-07-15,2177-07-15 04:00:00,2177-07-15,2177-07-15 20:00:00,NaN,32838,NaN,NaN,8.0,NaN,NaN,4132161,NaN,NaN,P000000,NaN,00245004101,44890917,PO,TAB,potassium chloride 10 MEQ Extended Release Ora...,Drug,RxNorm,Clinical Drug,S,628953,2006-06-04,2099-12-31,NaN,EHR prescription,Type Concept,Type Concept,Type Concept,S,OMOP4976911,2020-08-20,2099-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,19049105.0,10.0,9551.0,NaN,NaN,NaN,NaN,NaN,1970-01-01,2099-12-31,NaN,False,9551.0,milliequivalent,Unit,UCUM,Unit,S,10*-3.eq,1970-01-01,2099-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,potassium chloride,Drug,RxNorm,Ingredient,S,8591,1970-01-01,2099-12-31,NaN,"(None, None)",NaN,NaN,0
47,P000038,P000000,19049684,2177-07-15,2177-07-15 04:00:00,2177-07-15,2177-07-15 20:00:00,NaN,32838,NaN,NaN,6.0,NaN,NaN,4132161,NaN,NaN,P000000,NaN,00245004101,44890917,PO,TAB,potassium chloride 10 MEQ Extended Release Ora...,Drug,RxNorm,Clinical Drug,S,628953,2006-06-04,2099-12-31,NaN,EHR prescription,Type Concept,Type Concept,Type Concept,S,OMOP4976911,2020-08-20,2099-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,19049105.0,10.0,9551.0,NaN,NaN,NaN,NaN,NaN,1970-01-01,2099-12-31,NaN,False,9551.0,milliequivalent,Unit,UCUM,Unit,S,10*-3.eq,1970-01-01,2099-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,potassium chloride,Drug,RxNorm,Ingredient,S,8591,1970-01-01,2099-12-31,NaN,"(None, None)",-2.0,-33.33,-1
48,P000039,P000000,19049684,2177-07-15,2177-07-15 04:00:00,2177-07-15,2177-07-15 20:00:00,NaN,32838,NaN,NaN,4.0,NaN,NaN,4132161,NaN,NaN,P000000,NaN,00245004101,44890917,PO,TAB,potassium chloride 10 MEQ Extended Release Ora...,Drug,RxNorm,Clinical Drug,S,628953,2006-06-04,2099-12-31,NaN,EHR prescrip

In [59]:
fig = px.timeline(
    visit_df,
    x_start="drug_exposure_start_datetime",
    x_end="drug_exposure_end_datetime",
    y="concept_name",
    color="quantity_diff_trend",
    color_continuous_scale="bluered",
    opacity=0.5,
)
fig.show()

ValueError: Value of 'y' is not the name of a column in 'data_frame'. Expected one of ['drug_exposure_id', 'person_id', 'drug_concept_id', 'drug_exposure_start_date', 'drug_exposure_start_datetime', 'drug_exposure_end_date', 'drug_exposure_end_datetime', 'verbatim_end_date', 'drug_type_concept_id', 'stop_reason', 'refills', 'quantity', 'days_supply', 'sig', 'route_concept_id', 'lot_number', 'provider_id', 'visit_occurrence_id', 'visit_detail_id', 'drug_source_value', 'drug_source_concept_id', 'route_source_value', 'dose_unit_source_value', 'drug_concept_name', 'drug_domain_id', 'drug_vocabulary_id', 'drug_concept_class_id', 'drug_standard_concept', 'drug_concept_code', 'drug_valid_start_date', 'drug_valid_end_date', 'drug_invalid_reason', 'drug_type_concept_name', 'drug_type_domain_id', 'drug_type_vocabulary_id', 'drug_type_concept_class_id', 'drug_type_standard_concept', 'drug_type_concept_code', 'drug_type_valid_start_date', 'drug_type_valid_end_date', 'drug_type_invalid_reason', 'ro

In [32]:
drug_exposure_df.head(1)

,drug_exposure_id,person_id,drug_concept_id,drug_exposure_start_date,drug_exposure_start_datetime,drug_exposure_end_date,drug_exposure_end_datetime,verbatim_end_date,drug_type_concept_id,stop_reason,refills,quantity,days_supply,sig,route_concept_id,lot_number,provider_id,visit_occurrence_id,visit_detail_id,drug_source_value,drug_source_concept_id,route_source_value,dose_unit_source_value
0,P000000,P000000,40166274,2177-07-16,2177-07-16 22:00:00,2177-07-17,2177-07-17 21:00:00,NaN,32838,NaN,NaN,0.005,NaN,NaN,4142048,NaN,NaN,P000000,NaN,00002751001,45144375,SC,VIAL


In [35]:
person_id= "P000000"
drug_exposure_df[drug_exposure_df["person_id"] == person_id].shape
drug_exposure_df[drug_exposure_df["person_id"] == person_id]["visit_occurrence_id"].value_counts()

(434, 23)

visit_occurrence_id
P000000    104
P000049     58
P000112     55
P000155     51
P000188     48
P000192     45
P000038     40
P000115     33
Name: count, dtype: int64

In [36]:
patient_df = drug_exposure_df[drug_exposure_df["person_id"] == person_id]

In [12]:
patient_df[["drug_exposure_start_date", "drug_exposure_end_date"]].describe()
patient_df[["drug_exposure_start_datetime", "drug_exposure_end_datetime"]].describe()

,drug_exposure_start_date,drug_exposure_end_date
count,1182,1182
unique,93,92
top,2150-04-10,2147-11-30
freq,61,50


,drug_exposure_start_datetime,drug_exposure_end_datetime
count,1182,1182
unique,414,308
top,2150-04-10 07:00:00,2147-11-30 21:00:00
freq,36,50


In [13]:
patient_df.head()

,drug_exposure_id,person_id,drug_concept_id,drug_exposure_start_date,drug_exposure_start_datetime,drug_exposure_end_date,drug_exposure_end_datetime,verbatim_end_date,drug_type_concept_id,stop_reason,refills,quantity,days_supply,sig,route_concept_id,lot_number,provider_id,visit_occurrence_id,visit_detail_id,drug_source_value,drug_source_concept_id,route_source_value,dose_unit_source_value
653,-7496322014807298311,7155255168997124770,19078557,2147-09-12,2147-09-12 13:00:00,2147-09-12,2147-09-12 23:00:00,NaN,32838,NaN,NaN,10.0,NaN,NaN,4142048,NaN,NaN,-7759653971096571129,NaN,00002821501,45178530,SC,VIAL
654,1278734725911696306,7155255168997124770,19068781,2147-09-12,2147-09-12 11:00:00,2147-09-12,2147-09-12 23:00:00,NaN,32838,NaN,NaN,1.0,NaN,NaN,4167540,NaN,NaN,-7759653971096571129,NaN,00071015892,44924015,PO/NG,TAB
655,6359928042509081942,7155255168997124770,19071700,2147-09-12,2147-09-12 13:00:00,2147-09-12,2147-09-12 23:00:00,NaN,32838,NaN,NaN,1.0,NaN,NaN,4142048,NaN,NaN,-7759653971096571129,NaN,00088222033,45060408,SC,VIAL
656,6622361697988094299,7155255168997124770,19071700,2147-09-12,2147-09-12 13:00:00,2147-09-12,2147-09-12 23:00:00,NaN,32838,NaN,NaN,1.0,NaN,NaN,4142048,NaN,NaN,-7759653971096571129,NaN,00088222033,45060408,SC,VIAL
657,4999808883224663607,7155255168997124770,1593969,2147-09-12,2147-09-12 07:00:00,2147-09-12,2147-09-12 23:00:00,NaN,32838,NaN,NaN,1.0,NaN,NaN,4171047,NaN,NaN,-7759653971096571129,NaN,00338105102,45197779,IV DRIP,BTL


In [37]:
patient_df[["drug_exposure_start_date", "drug_exposure_start_datetime", "drug_exposure_end_date", "drug_exposure_end_datetime"]].sample(10)

,drug_exposure_start_date,drug_exposure_start_datetime,drug_exposure_end_date,drug_exposure_end_datetime
2562,2179-07-25,2179-07-25 03:00:00,2179-07-28,2179-07-28 20:00:00
14001,2177-09-05,2177-09-05 01:00:00,2177-09-06,2177-09-06 00:00:00
13833,2182-05-01,2182-05-01 12:00:00,2182-05-02,2182-05-02 15:00:00
8463,2178-09-29,2178-09-29 03:00:00,2178-10-02,2178-10-02 22:00:00
56,2177-07-15,2177-07-15 00:00:00,2177-07-15,2177-07-15 06:00:00
2540,2179-07-28,2179-07-28 07:00:00,2179-07-28,2179-07-28 20:00:00
52,2177-07-15,2177-07-15 05:00:00,2177-07-15,2177-07-15 20:00:00
80,2177-07-15,2177-07-15 08:00:00,2177-07-24,2177-07-24 17:00:00
62,2177-07-15,2177-07-15 04:00:00,2177-07-15,2177-07-15 20:00:00
13999,2177-09-04,2177-09-04 10:00:00,2177-09-07,2177-09-07 21:00:00


In [38]:
patient_df["visit_occurrence_id"].nunique()

8

In [39]:
patient_df["drug_exposure_start_datetime"] = patient_df["drug_exposure_start_datetime"].astype("datetime64[s]")
patient_df["drug_exposure_end_datetime"] = patient_df["drug_exposure_end_datetime"].astype("datetime64[s]")

/var/folders/5t/vb6260nn0dd8rm99r3ls2p8h0000gp/T/ipykernel_4190/3582365127.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient_df["drug_exposure_start_datetime"] = patient_df["drug_exposure_start_datetime"].astype("datetime64[s]")
/var/folders/5t/vb6260nn0dd8rm99r3ls2p8h0000gp/T/ipykernel_4190/3582365127.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient_df["drug_exposure_end_datetime"] = patient_df["drug_exposure_end_datetime"].astype("datetime64[s]")


In [40]:
patient_df["drug_exposure_start_datetime"].describe()

count                    434
mean     2178-11-08 00:26:16
min      2177-07-14 20:00:00
25%      2177-09-04 10:00:00
50%      2178-09-29 18:00:00
75%      2179-03-30 10:00:00
max      2182-05-03 11:00:00
Name: drug_exposure_start_datetime, dtype: object

In [41]:
patient_df["drug_concept_id"].nunique()
patient_df["drug_concept_id"].value_counts()

101

drug_concept_id
19071700    43
40166274    36
46275280    34
1596977     17
40221385    12
            ..
19072176     1
19029876     1
46287354     1
19077546     1
40167443     1
Name: count, Length: 101, dtype: int64

In [57]:
tmp = pd.concat([pd.Series(row.drug_concept_id, pd.date_range(row.drug_exposure_start_date, row.drug_exposure_end_date)) for row in patient_df.itertuples()])
tmp_df = pd.DataFrame(tmp, columns=["drug_concept_id"]).reset_index(names="date")
tmp_df.head()

,date,drug_concept_id
0,2147-09-12,19078557
1,2147-09-12,19068781
2,2147-09-12,19071700
3,2147-09-12,19071700
4,2147-09-12,1593969


In [51]:
tmp_drug = pd.concat([pd.Series(row.drug_concept_id, pd.date_range(row.drug_exposure_start_date, row.drug_exposure_end_date)) for row in patient_df.itertuples()])
tmp_drug_df = pd.DataFrame(tmp_drug, columns=["drug_concept_id"])
tmp_visit = pd.concat([pd.Series(row.visit_occurrence_id, pd.date_range(row.drug_exposure_start_date, row.drug_exposure_end_date)) for row in patient_df.itertuples()])
tmp_visit_df = pd.DataFrame(tmp_visit, columns=["visit_occurrence_id"])
tmp_df = pd.concat([tmp_drug_df, tmp_visit_df], axis=1).reset_index(names="date")

grouped_tmp_df = tmp_df.groupby("date").agg({"drug_concept_id": ["unique", "nunique"], "visit_occurrence_id": ["unique", "nunique"]}).reset_index()

# new_grouped_tmp_df = grouped_tmp_df[grouped_tmp_df[("visit_occurrence_id", "nunique")] == 1]
grouped_tmp_df.columns = ["date", "unique_drugs", "nb_drugs", "visit_id", "nb_visits"]
grouped_tmp_df["visit_id"] = grouped_tmp_df["visit_id"].apply(lambda x: x[0]).astype(str)

fig = px.scatter(grouped_tmp_df, x="date", y="nb_drugs", color="visit_id")
fig.show()

In [43]:
tmp_drug = pd.concat([pd.Series(row.drug_concept_id, pd.date_range(row.drug_exposure_start_date, row.drug_exposure_end_date)) for row in patient_df.itertuples()])
tmp_drug_df = pd.DataFrame(tmp_drug, columns=["drug_concept_id"])
tmp_drug_df.head()
tmp_visit = pd.concat([pd.Series(row.visit_occurrence_id, pd.date_range(row.drug_exposure_start_date, row.drug_exposure_end_date)) for row in patient_df.itertuples()])
tmp_visit_df = pd.DataFrame(tmp_visit, columns=["visit_occurrence_id"])
tmp_visit_df.head()
tmp_df = pd.concat([tmp_drug_df, tmp_visit_df], axis=1).reset_index(names="date")
tmp_df.head()

,drug_concept_id
2177-07-16,40166274
2177-07-17,40166274
2177-07-17,40166274
2177-07-18,40166274
2177-07-15,40166274


,visit_occurrence_id
2177-07-16,P000000
2177-07-17,P000000
2177-07-17,P000000
2177-07-18,P000000
2177-07-15,P000000


,date,drug_concept_id,visit_occurrence_id
0,2177-07-16,40166274,P000000
1,2177-07-17,40166274,P000000
2,2177-07-17,40166274,P000000
3,2177-07-18,40166274,P000000
4,2177-07-15,40166274,P000000


In [85]:
tmp_visit_df[tmp_visit_df["visit_occurrence_id"] == 2717440401888979354].sort_index()

,visit_occurrence_id
2148-06-24,2717440401888979354
2148-06-24,2717440401888979354
2148-06-24,2717440401888979354
2148-06-24,2717440401888979354
2148-06-24,2717440401888979354
...,...
2148-06-28,2717440401888979354
2148-06-28,2717440401888979354
2148-06-30,2717440401888979354
2148-07-03,2717440401888979354


In [44]:
tmp_df[tmp_df["date"] == dt.datetime(2148, 6, 30)]

,date,drug_concept_id,visit_occurrence_id


In [45]:
grouped_tmp_df = tmp_df.groupby("date").agg({"drug_concept_id": ["unique", "nunique"], "visit_occurrence_id": ["unique", "nunique"]}).reset_index()
grouped_tmp_df.head()

date                                    drug_concept_id          \
                                                         unique nunique   
0 2177-07-14  [42708743, 40221369, 1718698, 19074247, 190201...       7   
1 2177-07-15  [40166274, 19078557, 1549218, 36250141, 154599...      42   
2 2177-07-16  [40166274, 1549218, 36250141, 1545999, 1907170...      33   
3 2177-07-17  [40166274, 1549218, 36250141, 1545999, 1907170...      34   
4 2177-07-18  [40166274, 1549218, 36250141, 1545999, 1907170...      34   

  visit_occurrence_id          
               unique nunique  
0           [P000000]       1  
1           [P000000]       1  
2           [P000000]       1  
3           [P000000]       1  
4           [P000000]       1

In [46]:
grouped_tmp_df[("visit_occurrence_id", "nunique")].value_counts()

(visit_occurrence_id, nunique)
1    46
Name: count, dtype: int64

In [47]:
grouped_tmp_df[grouped_tmp_df[("visit_occurrence_id", "nunique")] == 2]

Empty DataFrame
Columns: [(date, ), (drug_concept_id, unique), (drug_concept_id, nunique), (visit_occurrence_id, unique), (visit_occurrence_id, nunique)]
Index: []

In [89]:
visit_occurrence_df.head(1)
visit_detail_df.head(1)

,visit_occurrence_id,person_id,visit_concept_id,visit_start_date,visit_start_datetime,visit_end_date,visit_end_datetime,visit_type_concept_id,provider_id,care_site_id,visit_source_value,visit_source_concept_id,admitting_source_concept_id,admitting_source_value,discharge_to_concept_id,discharge_to_source_value,preceding_visit_occurrence_id
0,-4406053801395356975,4783904755296699562,38004207,2112-11-06,2112-11-06 11:05:00,2112-11-06,2112-11-06 11:05:00,32817,NaN,NaN,10035631|2112-11-06,2000001801,NaN,NaN,NaN,NaN,-3.100296e+18


,visit_detail_id,person_id,visit_detail_concept_id,visit_detail_start_date,visit_detail_start_datetime,visit_detail_end_date,visit_detail_end_datetime,visit_detail_type_concept_id,provider_id,care_site_id,admitting_source_concept_id,discharge_to_concept_id,preceding_visit_detail_id,visit_detail_source_value,visit_detail_source_concept_id,admitting_source_value,discharge_to_source_value,visit_detail_parent_id,visit_occurrence_id
0,-1757828362327778468,3129727379702505063,8870,2197-04-17,2197-04-17 09:48:00,2197-04-17,2197-04-17 11:44:19,32817,NaN,-3.633440e+18,8870.0,NaN,NaN,10002930|25282382|38481760,2000001903,EMERGENCY ROOM,NaN,NaN,-9127810274408915712


In [86]:
patient_df[patient_df["visit_occurrence_id"] == 2717440401888979354].sort_values(by=["drug_exposure_start_date", "drug_exposure_end_date"])

,drug_exposure_id,person_id,drug_concept_id,drug_exposure_start_date,drug_exposure_start_datetime,drug_exposure_end_date,drug_exposure_end_datetime,verbatim_end_date,drug_type_concept_id,stop_reason,refills,quantity,days_supply,sig,route_concept_id,lot_number,provider_id,visit_occurrence_id,visit_detail_id,drug_source_value,drug_source_concept_id,route_source_value,dose_unit_source_value
12171,-300938841025473319,7155255168997124770,1113346,2148-06-24,2148-06-24 18:00:00,2148-06-25,2148-06-25 15:00:00,NaN,32838,NaN,NaN,1.0,NaN,NaN,4167540,NaN,NaN,2717440401888979354,NaN,00904628889,44910513,PO/NG,TAB
12197,817507700258432499,7155255168997124770,19019112,2148-06-24,2148-06-24 19:00:00,2148-06-25,2148-06-25 18:00:00,NaN,32838,NaN,NaN,1.0,NaN,NaN,4167540,NaN,NaN,2717440401888979354,NaN,63739049910,45108668,PO/NG,TAB
12145,-6619715848743895892,7155255168997124770,40166274,2148-06-24,2148-06-24 18:00:00,2148-06-26,2148-06-26 12:00:00,NaN,32838,NaN,NaN,1.0,NaN,NaN,4142048,NaN,NaN,2717440401888979354,NaN,00002751001,45144375,SC,VIAL
12155,-2999314229284571782,7155255168997124770,19071700,2148-06-24,2148-06-24 22:00:00,2148-06-26,2148-06-26 12:00:00,NaN,32838,NaN,NaN,1.0,NaN,NaN,4142048,NaN,NaN,2717440401888979354,NaN,00088222033,45060408,SC,VIAL
12156,9220948588785381346,7155255168997124770,19071700,2148-06-24,2148-06-24 08:00:00,2148-06-26,2148-06-26 12:00:00,NaN,32838,NaN,NaN,1.0,NaN,NaN,4142048,NaN,NaN,2717440401888979354,NaN,00088222033,45060408,SC,VIAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12202,-3519568063102425770,7155255168997124770,40164492,2148-06-27,2148-06-27 08:00:00,2148-06-28,2148-06-28 18:00:00,NaN,32838,NaN,NaN,1.0,NaN,NaN,4167540,NaN,NaN,2717440401888979354,NaN,64764011907,45176946,PO/NG,TAB
12152,-168353688812238794,7155255168997124770,1551170,2148-06-28,2148-06-28 08:00:00,2148-06-28,2148-06-28 18:00:00,NaN,32838,NaN,NaN,2.0,NaN,NaN,4132161,NaN,NaN,2717440401888979354,NaN,00054001820,44957599,PO,TAB
12198,-4543917500577915795,7155255168997124770,1551101,2148-06-30,2148-06-30 08:00:00,2148-06-30,2148-06-30 08:00:00,NaN,32838,NaN,NaN,3.0,NaN,NaN,4132161,NaN,NaN,2717440401888979354,NaN,63739051910,45125502,PO,TAB
12150,6846587631550174607,7155255168997124770,1551170,2148-07-03,2148-07-03 08:00:00,2148-07-03,2148-07-03 08:00:00,NaN,32838,NaN,NaN,1.0,NaN,NaN,4132161,NaN,NaN,2717440401888979354,NaN,00054001820,44957599,PO,TAB


In [80]:
visit_occurrence_df[(visit_occurrence_df["person_id"] == person_id) & (visit_occurrence_df["visit_occurrence_id"].isin([2717440401888979354, -2408772910912676912]))]

,visit_occurrence_id,person_id,visit_concept_id,visit_start_date,visit_start_datetime,visit_end_date,visit_end_datetime,visit_type_concept_id,provider_id,care_site_id,visit_source_value,visit_source_concept_id,admitting_source_concept_id,admitting_source_value,discharge_to_concept_id,discharge_to_source_value,preceding_visit_occurrence_id
683,2717440401888979354,7155255168997124770,581385,2148-06-24,2148-06-24 09:22:00,2148-06-28,2148-06-28 13:54:00,32817,NaN,NaN,10014354|23132022,2000001807,38004207.0,PHYSICIAN REFERRAL,581476.0,HOME,NaN
802,-2408772910912676912,7155255168997124770,262,2148-06-29,2148-06-29 21:06:00,2148-07-13,2148-07-13 19:35:00,32817,NaN,NaN,10014354|27487226,2000001809,8717.0,TRANSFER FROM HOSPITAL,581476.0,HOME HEALTH CARE,NaN


In [76]:
patient_df[patient_df["visit_occurrence_id"].isin([2717440401888979354, -2408772910912676912])]

,drug_exposure_id,person_id,drug_concept_id,drug_exposure_start_date,drug_exposure_start_datetime,drug_exposure_end_date,drug_exposure_end_datetime,verbatim_end_date,drug_type_concept_id,stop_reason,refills,quantity,days_supply,sig,route_concept_id,lot_number,provider_id,visit_occurrence_id,visit_detail_id,drug_source_value,drug_source_concept_id,route_source_value,dose_unit_source_value
12143,-7417655728750480613,7155255168997124770,40166274,2148-06-26,2148-06-26 13:00:00,2148-06-26,2148-06-26 13:00:00,NaN,32838,NaN,NaN,1.0,NaN,NaN,4142048,NaN,NaN,2717440401888979354,NaN,00002751001,45144375,SC,VIAL
12144,565603239085007381,7155255168997124770,40166274,2148-06-26,2148-06-26 17:00:00,2148-06-27,2148-06-27 12:00:00,NaN,32838,NaN,NaN,1.0,NaN,NaN,4142048,NaN,NaN,2717440401888979354,NaN,00002751001,45144375,SC,VIAL
12145,-6619715848743895892,7155255168997124770,40166274,2148-06-24,2148-06-24 18:00:00,2148-06-26,2148-06-26 12:00:00,NaN,32838,NaN,NaN,1.0,NaN,NaN,4142048,NaN,NaN,2717440401888979354,NaN,00002751001,45144375,SC,VIAL
12146,-9004465727595887306,7155255168997124770,40166274,2148-06-26,2148-06-26 13:00:00,2148-06-28,2148-06-28 18:00:00,NaN,32838,NaN,NaN,1.0,NaN,NaN,4142048,NaN,NaN,2717440401888979354,NaN,00002751001,45144375,SC,VIAL
12147,-1011315529531470722,7155255168997124770,19078557,2148-06-26,2148-06-26 14:00:00,2148-06-27,2148-06-27 12:00:00,NaN,32838,NaN,NaN,10.0,NaN,NaN,4142048,NaN,NaN,2717440401888979354,NaN,00002821501,45178530,SC,VIAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15706,-7827247298232247086,7155255168997124770,19127213,2148-06-30,2148-06-30 08:00:00,2148-07-13,2148-07-13 23:00:00,NaN,32838,NaN,NaN,1.0,NaN,NaN,4171047,NaN,NaN,-2408772910912676912,NaN,Sodium Chloride 0.9% Flush 10 mL Syringe,2000011396,IV,SYR
15707,6204318315453781902,7155255168997124770,19127213,2148-07-10,2148-07-10 19:00:00,2148-07-13,2148-07-13 23:00:00,NaN,32838,NaN,NaN,0.3,NaN,NaN,4171047,NaN,NaN,-2408772910912676912,NaN,Sodium Chloride 0.9% Flush 10 mL Syringe,2000011396,IV,SYR
15708,-2535157145593732078,7155255168997124770,19127213,2148-07-07,2148-07-07 17:00:00,2148-07-08,2148-07-08 13:00:00,NaN,32838,NaN,NaN,0.3,NaN,NaN,4171047,NaN,NaN,-2408772910912676912,NaN,Sodium Chloride 0.9% Flush 10 mL Syringe,2000011396,IV,SYR
15709,8004527106975967806,7155255168997124770,19127213,2148-07-04,2148-07-04 21:00:00,2148-07-08,2148-07-08 13:00:00,NaN,32838,NaN,NaN,0.3,NaN,NaN,4171047,NaN,NaN,-2408772910912676912,NaN,Sodium Chloride 0.9% Flush 10 mL Syringe,2000011396,IV,SYR


In [48]:
new_grouped_tmp_df = grouped_tmp_df[grouped_tmp_df[("visit_occurrence_id", "nunique")] == 1]
new_grouped_tmp_df.columns = ["date", "unique_drugs", "nb_drugs", "visit_id", "nb_visits"]
new_grouped_tmp_df["visit_id"] = new_grouped_tmp_df["visit_id"].apply(lambda x: x[0]).astype(str)
new_grouped_tmp_df.head(1)

,date,unique_drugs,nb_drugs,visit_id,nb_visits
0,2177-07-14,"[42708743, 40221369, 1718698, 19074247, 190201...",7,P000000,1


In [49]:
fig = px.scatter(new_grouped_tmp_df, x="date", y="nb_drugs", color="visit_id")
fig.show()

In [32]:
tmp_df.head()

,date,drug_concept_id
0,2147-09-12,19078557
1,2147-09-12,19068781
2,2147-09-12,19071700
3,2147-09-12,19071700
4,2147-09-12,1593969


In [38]:
a = tmp_df[tmp_df["drug_concept_id"] == 40166274]
fig = px.scatter(x=a["date"], y=a["drug_concept_id"])
fig.show()

In [17]:

fig = px.line(patient_df, x="drug_exposure_start_datetime", y="quantity")
fig.show()

In [53]:
f = px.scatter()
f.show()